<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*This notebook contains an excerpt from the [Python Business Automation Handbook](https://smartai1004.tistory.com/46?category=946283) by Smart ICT; 
the content is available [on GitHub](https://github.com/aiml1004/automate_basic).*

*본 자료를 무단 복제, 배포하는 경우에는 저작권법 제97조의 저작재산권침해죄에 해당하여 저작권법에 의거 법적 조치에 처해질 수 있습니다.*
*COPYRIGHT ⓒ 2020 SMART ICT ALL RIGHTS RESERVED.* 

*If you find this content useful, please consider supporting the work by [buying the book](https://kmong.com/gig/233618)!*

<!--NAVIGATION-->
< [Preface](00.00-Preface.ipynb) | [Contents](00_Index.ipynb) | [Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb) >

## 시나리오

무엇을 하고 싶은가?  

과학기술정보통신부 사업공고 게시판의 내용을 확인해 새로운 게시글이 있다면 공공영업팀장 및 팀원들께 메일을 보낸다.   
메일은 매일 아침 8시에 자동으로 보낸다. (월~금 weekday에만 수행)  

과학기술정보통신부에 접속한다.   
사업공고 게시판의 게시글을 엑셀로 저장한다.   
어제와 다른 새로운 게시글(사업공고)이 있는지 확인한다.   
새로운 게시글의 첨부문서를 다운로드한다.   
오늘 다운로드한 게시글을 엑셀로 저장한다.   

영업활동에 참고하도록 영업팀장님께 메일로 보낸다.    
* 오늘 다운로드한 게시글 엑셀   
* 신규과제 공고문(신청요강 포함), 연구개발계획서(양식) 등 첨부파일은 사업별로 압축파일을 만든다.   

과학기술정보통신부 홈페이지 사업공고 게시판에 가서

사업공고 목록 게시판의 내용을 엑셀로 저장한다.  : 인터넷 크롤링, 파이썬 엑셀 처리
새로운 사업공고 게시글(포스팅) 내역을 찾는다. : 엑셀 내용을 비교해 새로운 게시글 찾기
사업공고 상세 내역 게시판에서 첨부문서를 다운로드 하여 정리한다.  : 파일처리
매일 아침 8시에 자동으로 메일을 보낸다.  : 작업 스케줄링 자동화

In [ ]:
# selenium.webdriver 모듈은 모든 WebDriver의 구현을 제공합니다. 현재 지원되는 WebDriver 구현은
# Firefox, Chrome, IE를 지원한다.
# 장치 드라이버는 하드웨어를 제어하기 위한 소프트웨어이다.
# Web Driver는 웹(인터넷브라우저)을 제어하기 위한 소프트웨어이다.

## 전체 구조   
전체 구조를 이야기 하고 강의를 시작하는 것이 좋겠다.  
예를 이런 흐름으로 코딩을 해봅시다

## 설치 필요 사항

### chromedriver 다운로드 방법 가이드

* 크롬 버전에 맞는 드라이버를 선택   
* 크롬 버전 보는법   
   크롬 > 더보기 > 도움말 > Chrome 정보(G)

다운로드 방법   
https://blog.naver.com/song_sec/221752226329

### 크롬브라우저를 설치 하세요.  

### 제약조건 얘기
MS 윈도우를 기준으로 작성 했습니다. 제가 맥 사용경험이 없습니다.  
다만 리눅스/유닉스에 경험이 있으니 유닉스를 기준으로 말씀드릴수는 있습니다.  

In [ ]:
# selenium의 파일 다운로드 경로 변경하기
# https://cozynow.tistory.com/43

In [ ]:
# 02_사업공고_조회

In [1]:
# 얘네들이 뭐하는 애들인가? @@@
import pandas as pd
from time import sleep
import os, sys
import shutil, glob, time

In [2]:
#처음부터 아래와 같은 글로벌 변수를 아는 것은 어렵다.
#처음에는 기냥하다가 반복이 되면 위로 올라온다

### 글로벌 변수를 먼저 작성해놓으니 좀 뜬검없다.
왜 글로벌 변수를 사용하나?  
엑셀은 우리가 이 프로그램을 개발할때 사용하는 정보의 목록이다. 
귀찮아서. 손을 고생시키지 않기 위해서..

In [3]:
BASE_DIR = 'C:\\mylab\\book\\automate_basic'
BOARD_LIST_XL = BASE_DIR+'\\data\\board_list.xlsx'        # 게시판 디폴트 조회 목록
SEND_HIST_XL  = BASE_DIR+'\\data\\mail_send_hsitory.xlsx' # 메일을 보낸 게시물 목록
NEW_POST_LIST_XL = BASE_DIR+'\\data\\new_post_list.xlsx'  # 어제 메일을 보낸 이후 새로운 게시물 목록
DOWNLOAD_DIR = BASE_DIR+'\\download'   #첨부문서 다운로드 위치. 우리가 원하는 곳에 저장하기 위한 설정

In [6]:
# import webdriver
from selenium import webdriver
chromedriver = "C:\\mylab\\book\\chromedriver" 
driver = webdriver.Chrome(chromedriver)

In [7]:
# 네이버로 가볼가요.
# driver.????('https://www.naver.com/') 

# driver.get('https://www.naver.com/') 

[과학기술정보통신부 홈페이지](https://www.msit.go.kr/web/main/main.do)

## 게시판 게시글을 가져오려면?

[과학기술정보통신부 사업공고 게시판](https://www.msit.go.kr/web/msipContents/contents.do?mId=MTAzNQ==)

### 과학기술정보통신부 사업공고 게시판 HTML 분석

In [ ]:
# 무엇을 알고자 하는가? 

# 과학기술정보통신부 사업공고 목록 게시판 
# HTML 분석 목적: 어떤 tag / class로 찾으면 목록을 가져올수 있는가?

In [ ]:
# 어떻게 우리가 원하는 항목을 가져올까요?
# 게시판은 어떤식으로 반복되고 있나요? 
# tag / class는 식별하기에 적합한가요? 어떻게 적합한지 알수 있나요?

In [8]:
driver.get('https://www.msit.go.kr/web/msipContents/contents.do?mId=MTAzNQ==') # 이 URL은 어떻게 알게 되었는가?
# 알림> 사업공고 로 이동하니 위 url이 나오더라

### 개발자 도구로 HTML을 분석 해보자.   
개발자 도구를 F12를 눌러요.

In [ ]:
# driver.find_element_by_class_name?

In [9]:
# 'board_list' class element의 모든 하위 element(내용)을 가져오고 싶어요.
# 왜 class_name 인가?  find_element_by_class_name 어떻게 알았나요?  게시판 구조에 div class에 있었잖아요. 
#element = driver.find_??('??') 

element = driver.find_element_by_class_name('board_list') 

In [ ]:
# 어떻게 element의 내용(text)을 볼수 있을까요?

# element은 text 어트리뷰트를 가지고 있을까요? 
# element에 또 다른 어떤 어트리뷰트를 이용할수 있을까요? 

# 이렇게 찾은 element도 웹문서는 text가 있겠죠
# element이 WebElement 이니 text가 있겠죠.  그러면 text를 한번 확인해보죠.

# dir(element) 
"""
 'find_element_by_class_name',
 'find_element_by_css_selector',
 ...생략,
 'submit',
 'tag_name',
 'text',
 'value_of_css_property
"""

In [ ]:
# 내용을 확인해봐요. 무슨 내용을 확인하나요?  .text가 있다는 것을 어떻게 알았나요?

In [11]:
# 그러면 text를 한번 확인해보죠.
# 게시판의 목록을 담고 있는 속성(변수)이 맞나요?

element.text  #보통은 tab 키를 눌러 자동완성 하겠지요.

'1073\n2020년도 EU 협력진흥사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.\n1072\n2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.\n1071\n2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모\n부서 공공기술기반팀 : 허대성 연락처 044-202-4639\n첨부파일 2020.09.28.\n1070\n2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고\n부서 공공기술기반팀 : 허대성 연락처 044-202-4639\n첨부파일 2020.09.18.\n1069\n2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모\n부서 기초연구진흥과 : 오늘 연락처 044-202-4538\n첨부파일 2020.09.10.\n1068\n2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고\n부서 정보통신산업기반과 : 신유진 연락처 044-202-6245\n첨부파일 2020.09.10.\n1067\n2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모\n부서 공공기술기반팀 : 허대성 연락처 044-202-4639\n첨부파일 2020.09.10.\n1066\n2020년도 재난안전플랫폼기술개발사업 신규과제 선정결과 공고\n부서 공공기술기반팀 : 허대성 연락처 044-202-4639\n첨부파일 2020.09.07.\n1065\n2020년도 국민생활안전 긴급대응연구사업 신규과제 선정결과 공고\n부서 공공기술기반팀 : 허대성 연락처 044-202-4639\n첨부파일 2020.09.07.\n1064\n2020년도 제2차 해외고급과학자초빙사업 Plus (Brain Pool+) 연장공고\n부서 미래인재양성과 : 양성녀 연락처 044-202-4836\n첨부파일 2020.09.01.'

In [ ]:
# 한개의 row 만 보면 아래와 같네요.

# 1073\n2020년도 EU 협력진흥사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.

## 미션은 게시판 조회결과를 엑셀로 다운로드 받는 것이에요

In [12]:
# 왜 a_tag 인가요?  a_tag를 여러번 반복하면서 목록을 표현하고 있기 때문에
# Returns: - list of WebElement
# elements가 복수형이다
# a_tag_list = element.find_elements~~

a_tag_list = element.find_elements_by_tag_name('a') 

In [ ]:
# 아래와 깉이 출력 해보세요
# 게시글 수는 10개 입니다
# print("게시글 수는 " + len(a_tag_list) + "개 입니다")

print("게시글 수는 " + str(len(a_tag_list)) +"개 입니다")

In [14]:
# a_tag_list도 text를 가지고 있지 않을까요?

# a_tag_list.text

In [15]:
# 첫번째 게시글 a_tag의 text를 출력 해보세요.  
# 파이썬 리스트인덱스는 0 부터 시작한다
# a_tag_list[?].text  

a_tag_list[0].text 

'1073\n2020년도 EU 협력진흥사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.'

In [ ]:
# 두번째 게시글 a_tag의 text를 출력 해보세요.  


## 첫번째 게시글 내용을 항목별로 출력

In [16]:
# 타이핑하기 힘드니 a_tag_list[0]의 내용을 a_tag변수에 저장 
a_tag = a_tag_list[0]

In [19]:
# 게시글 제목을 출력
# <span class="title">2020년도 EU 협력진흥사업 신규과제 공모</span>
# title= a_tag.find_element_

title= a_tag.find_element_by_class_name("title")
print('title: ', title.text)

title:  2020년도 EU 협력진흥사업 신규과제 공모


## 부서와 연락처를 추출

In [21]:
#부서_담당자를 가져오시죠.
#<span class="con">구주아프리카협력담당관 : 이세원</span>
#contact = a_tag.find_element???

contact = a_tag.find_element_by_class_name("con")
print('부서_담당자: ', contact.text)

부서_담당자:  구주아프리카협력담당관 : 이세원


In [24]:
#전화번호를 가져오시죠.  # 문제점: span class 이름이 con으로 동일하네요. 어쩌죠? 
# 이것을 사용하면 안된다. 전화번호를 가져올수 없다.
contact = a_tag.find_element_by_class_name("con") 
print('전화번호: ', contact.text)

전화번호:  구주아프리카협력담당관 : 이세원


In [25]:
# "con"으로 된 모든 elements를 찾으시죠.
# <span class="con">구주아프리카협력담당관 : 이세원</span>
# <span class="con"> 044-202-4639</span>
# 모두 찾을 때는 복수형인 메소드(elements)를 활용
# contact_lst= a_tag.find_???

contact_lst= a_tag.find_elements_by_class_name("con")

In [26]:
# contact_lst list이고 WebElement 2개를 담고 있겠죠.
# 0번째는 담당자가 1번째는 전화번호가 저장되어 있다. 
contact_lst[0].text 

'구주아프리카협력담당관 : 이세원'

In [27]:
contact_lst[1].text

'044-202-4354'

In [29]:
# 게시일자를 출력하세요
date= a_tag.find_element_by_class_name('date') #element 단수형
date.text

'2020.09.29.'

In [ ]:
# 마지막에 .이 있다. .을 제거한다.
date.text[0:10] 

In [ ]:
# 고유번호로 사용할 만한 것은 무엇인가? 사람의 경우는 주민번호와 같은
# 한 개의 게시물을 고유하게 식별할 만한 것이 있는가? 
# 이것으로 어제 올라온 게시물과 비교해 새로운 게시물인지 아는데 활용한다
# artId가 고유번호인것 같다. 만약 저런 것이 없다면 사업명으로 구분할수 밖에 없을것 같다.

In [30]:
# <a href="/web/msipContents/contentsView.do?cateId=_tstb17&amp;artId=3114914">
u = a_tag.get_attribute('href') # 키보드 치기 싫어서 짧은 변수에 담는다. 어차피 지울 거라
u 

'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914'

In [ ]:
# 'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914' 에서 
# 3114914 이 번호만 추출하고 싶어요. 

#### 아래 쉬운 예를 별도 파일로 빼는게 좋을까요 @@@
문서 링크로 변경하는게 좋을 듯

In [38]:
# 먼저 간단한 예제부터 해보시죠. 
st = "012345"

'01'

In [42]:
#"012345" 라는 문자열에서 "01" 만 추출하세요
st[0:2]

'01'

In [44]:
#"012345" 라는 문자열에서 "12" 만 추출하세요
st[1:3]

'12'

In [45]:
#"012345" 라는 문자열에서 "34" 만 추출하세요
st[3:5]

'34'

In [46]:
u

'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914'

In [31]:
# 본론으로 돌아가, artId를 추출하고 하세요 
# 먼저 &artId= 문자들이 시작하는 위치를 찾아요
f_str = "&artId="
u.find(f_str)

70

In [32]:
# url의 전체 길이는?
len(u)

84

In [33]:
# 70 ~ 84 에 담긴 내용은 무엇일까요?
u[70:84]

'&artId=3114914'

In [34]:
# 찾는 문자열("&artId=")을 포함하고 있으니, 길이를 구하세요  
len(f_str)

7

In [35]:
# len(f_str) 만큼 더해준 길이 부터 시작하면 되겠네요.
u[70 + len(f_str) : 84]

'3114914'

In [36]:
#상수를 변수로 변경해 보시죠
#위에서 70, 84 이렇게 정해진 값을 상수라고 해요.
u[u.find(f_str) + len(f_str) : len(u)]

'3114914'

In [37]:
# u는 나중에 알아보기 힘드니, url로 바꾸세요
# f_str --> find_str로 변수명을 변경
url = u
find_str =f_str
print("최종 : ", url[url.find(find_str) + len(find_str) : len(url)])

최종 :  3114914


In [47]:
# a_tag_list의 내용이 기억이 안나네요. 
# 게시글의 각 row를 WebElement에 담고 있네요.
a_tag_list  

[<selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="115a8357-990f-4188-9b09-c2bb1ee59823")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="5ca6358a-c5f3-446f-a930-dbebac65c0e6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="43a2649b-e4ec-45a8-b07c-6fd6d291431f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="a86dd20c-0cee-476b-82a7-d77b4bb3e7af")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="c8841c62-f0a5-4b2d-8fca-241bbc5aa37e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="255a50e2-a510-4e62-9adc-94b332fe0d89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="dd137e2b-279f-4af9-b03b-ab

In [48]:
# 위에서 테스트한 문장을 정리한다. 
# a_tag_list에서 한줄씩 꺼낸다(to a_tag).

rows = [] #게시글을 담을 리스트형 변수 
for a_tag in a_tag_list:   
    #print(a_tag.text)
    title= a_tag.find_element_by_class_name('title')
    contact_lst= a_tag.find_elements_by_class_name('con')
    date= a_tag.find_element_by_class_name('date')
    url = a_tag.get_attribute('href')    
    
    find_str = "&artId="
    post_id = url[url.find(find_str) + len(find_str) : len(url)] #게시글 고유id
    
    row = (title.text, contact_lst[0].text, contact_lst[1].text, date.text[0:10], url, post_id)
    rows.append(row) # 리스트 행추가

In [49]:
#첫번째 로우를 출력하세요
print(rows[0])
# print(rows[1])

('2020년도 EU 협력진흥사업 신규과제 공모', '구주아프리카협력담당관 : 이세원', '044-202-4354', '2020.09.29', 'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914', '3114914')


In [ ]:
# 테이블 형태로 출력하세요
# 파이선의 엑셀이 pandas DataFrame 입니다. # 이걸 사용해야 하는지 어떻게 알았나요?
# 얘를 이용해서 테이블형태로 출력해볼게요

In [50]:
# 테이블(DataFrame)의 열이름을 지정해요. # 엑셀의 열 이름과 동일

# 참고 > 한개의 row
# '2020년도 EU 협력진흥사업 신규과제 공모', '구주아프리카협력담당관 : 이세원', '044-202-4354', '2020.09.29', 
#  'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914', '3114914'
#  '사업명', '공고_담당자', '전화번호', '공고일', 'URL', 'post_id'
columns = ['사업명', '공고_담당자', '전화번호', '공고일', 'URL', 'post_id']  

In [51]:
df = pd.DataFrame(rows, columns=columns) #엑셀 워크시트 만들기와 유사

In [52]:
# 내용이 잘 만들어 졌을까요? 확인해요
df

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961
5,2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고,정보통신산업기반과 : 신유진,044-202-6245,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088727
6,2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088726
7,2020년도 재난안전플랫폼기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081711
8,2020년도 국민생활안전 긴급대응연구사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081710
9,2020년도 제2차 해외고급과학자초빙사업 Plus (Brain Pool+) 연장공고,미래인재양성과 : 양성녀,044-202-4836,2020.09.01,https://www.msit.go.kr/web/msipContents/conten...,3067835


In [53]:
BOARD_LIST_XL

'C:\\mylab\\book\\automate_basic\\data\\board_list.xlsx'

In [57]:
# 엑셀에 저장하기
# 저장위치: BOARD_LIST_XL
# encoding='cp949', index=False

df.to_excel(BOARD_LIST_XL, encoding='cp949', index=False)  

cp949, euc-kr 소개   
https://namu.wiki/w/CP949

In [58]:
# index=False를 했는가? @@@

## 사업공고 상세페이지의 첨부파일을 다운로드 한다

### before

In [59]:
#엑셀에서 읽어오기
df = pd.read_excel(BOARD_LIST_XL)

In [60]:
# 파일의 앞부분 5개만 미리보기 해요
df.head()

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961


In [61]:
# 너무 많은 데이터로 테스트를 하면 시간이 많이 걸려요. 다운로드 시간, 페이지 이동시간 등
# 2 개만 지정해서 테스트를 해요

In [62]:
# in 연산자로 필터링 해보세요. 조건 부합 데이터만 추출
query_str = "사업명 in ['2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모', '2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모']"  
df1 = df.query(query_str)     
df1

,사업명,공고_담당자,전화번호,공고일,URL,post_id
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773


In [63]:
# == 연산자로 필터링 해보세요, 위와 동일한 표현식이에요
query_str = "사업명 == ['2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모', '2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모']" 
df = df.query(query_str)     # 조건 부합 데이터 추출
df

,사업명,공고_담당자,전화번호,공고일,URL,post_id
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773


In [64]:
# df.itertuples? 
# Iterate over DataFrame rows as namedtuples.
# namedtuple이란?

In [65]:
# 데이터프레임에 저장된 URL을 활용해 각 게시물들의 첨부파일을 다운로드 해요
# dataframe 칼럼명과 동일한 칼럼명을 가진 튜플을 리턴(반환)한다.
for row in df.itertuples(index=True): 
    print(row.URL)

https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114913
https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3122773


In [66]:
# for 문의 마지막 row의 데이터를 확인해 보아요.

row 

Pandas(Index=2, 사업명='2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모', 공고_담당자='공공기술기반팀 : 허대성', 전화번호='044-202-4639', 공고일='2020.09.28', URL='https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3122773', post_id=3122773)

In [ ]:
# Index=2,사업명='2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모, ....
# dataframe의 칼럼명 = 값(value) 형태로 매핑되어 있다.  칼럼명은 다양한 이름으로 불린다. attribute(속성), 변수(variable) 등

In [68]:
#아래 출력결과를 클릭하면 사업공고 상세페이지가 열린다.
print(row.URL)

https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3122773


In [70]:
#저 url(사업공고 상세페이지)로 이동해요
#우리 미션은 다운로드 버튼을 클릭해서 첨부파일을 다운로드 받는 것이에요
# driver.???(row.URL)

driver.get(row.URL)

### 어떤 class/ tag 로 element를 찾아야 하나요?

In [71]:
#원하는 element를 가져 오세요.
# <ul class="down_file"> 
# element = driver.find_element???("???")

element = driver.find_element_by_class_name("down_file")

In [72]:
element.text

'(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp\n다운로드 바로보기\n(붙임2) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서(양식).zip\n다운로드 바로보기\n(붙임3) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서 기타 증빙(양식).hwp\n다운로드 바로보기\n(붙임4) 2020년도 국민생활안전 긴급대응연구사업 관련 주요 규정 및 지침.hwp\n다운로드 바로보기'

In [73]:
element2 = element.find_element_by_class_name('down_btn')

In [75]:
a_tag_list = element2.find_elements_by_tag_name('a')

In [76]:
a_tag_list[0].text

'다운로드'

In [77]:
a_tag_list[1].text

'바로보기'

In [78]:
#우리는 다운로드에 관심이 있다
a_tag = a_tag_list[0] 

### 왜 get_attribute 인가?

In [79]:
# url을 출력하세요. 다운로드에 필요한 url이 있나요? 
# 보통 'href'attribute에 url 있습니다
# my_url = a_tag.get_attribute(???)

my_url = a_tag.get_attribute('href')
print(my_url)  

javascript:void(0);


In [ ]:
# 아래 처럼, url링크가 아닌 자바스크립트가 들어가있는 경우에는 
# click() 명령을 하더라도  href 속성 값에 url 주소가 들어가 있지 않기 때문에 해당 주소로 이동할수 없어요.
# 이런 경우에는, onlclick 내부의 명령어가 실행되도록 해야죠. 

# <a href = "javascript:void(0);", onclick = "자바스크립트 명령어"> 다운로드 </a>

In [81]:
# onclick attribute에 저장된 "자바스크립트 명령어"가 필요하다.
# 이것을 호출해야 첨부파일이 다운로드 된다.
# a_tag.get_???('???')  

a_tag.get_attribute('onclick')  

"filePathEncode('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/28/(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp')"

In [ ]:
# 우리가 원하는 자바스크립트문장은 0번(a_tag_list[0] )에 들어 있다.  
# 첨부파일 다운로드 onclick에 지정된 문장은 filePathEncode로 시작한다.

In [82]:
# a_tag_list의 모든 a_tag의 onclick attribute 내용을 모두 출력하세요 
# for a_tag in a_tag_list: 
#     print(a_tag.??? )
#     print("")
    
for a_tag in a_tag_list: 
    print(a_tag.get_attribute('onclick'))
    print("")

filePathEncode('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/28/(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp')

getExtension_path('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/28/(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp')



In [83]:
# filePathEncode로 시작하는 문장만 출력하세요.
# 첨부파일 다운로드 onclick에 지정된 자바스크립트문장은 filePathEncode로 시작한다

# for a_tag in a_tag_list:
#     txt = a_tag.get_attribute('onclick')
#     if "???" in txt:  
#         print(txt)    
        
for a_tag in a_tag_list:
    txt = a_tag.get_attribute('onclick')
    if "filePathEncode" in txt:  
        print(txt)    

filePathEncode('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/28/(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp')


In [84]:
a_tag

<selenium.webdriver.remote.webelement.WebElement (session="807520c141192b9b94aa2a97750af91e", element="9c425f43-fad8-40d7-95cf-8e5164d3772c")>

### 첨부문서 다운로드

In [86]:
# 첨부문서 다운로드 하세요
# driver.execute_script
js_script ="""filePathEncode('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/28/(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp')"""
driver.execute_script(js_script, a_tag)  # 자바 명령어 실행   @@@execute_script 이걸 어떻게 알았는가?
# sleep(1)

In [154]:
# 첨부문서가 제대로 다운로드가 되는지 확인해 보자
# C:\Users\ME\Downloads 에 다운로드가 된다.

!dir C:\Users\ME\Downloads /b

(붙임1) 2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모 공고문.hwp


### refactoring_after

#### download default_directory 설정

In [90]:
#C:\Users\ME\Downloads에 다운로드하면 헤깔린다. 그래서 별도의 다운로드 폴도에 다운로드한다.

In [91]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()

In [92]:
DOWNLOAD_DIR

'C:\\mylab\\book\\automate_basic\\download'

In [93]:
chromedriver

'C:\\mylab\\book\\chromedriver'

In [95]:
# chrome_options.add_experimental_option?

download default_directory 설정 참고  
https://stackoverflow.com/questions/46937319/how-to-use-chrome-webdriver-in-selenium-to-download-files-in-python

In [96]:
# Adds an experimental option which is passed to chrome.
# prefs : to specify preferences that will be applied after Chrome starts
prefs = {"download.default_directory" : DOWNLOAD_DIR,
         "download.prompt_for_download": False 
        } 
chrome_options.add_experimental_option("prefs",prefs)  
driver = webdriver.Chrome(executable_path=chromedriver, options=chrome_options)

## 신규 게시글의 첨부파일만 다운로드 한다

### before

### 첨부 파일 저장 위치 테스트

In [97]:
# 첨부 파일 저장 위치 테스트
biz_name="2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모"
biz_name= biz_name.replace(" ","_")
biz_name

'2020년도_제4차_국민생활안전_긴급대응연구사업_신규과제_재공모'

In [103]:
# "."을 "\\"로 바꾸세요
# date='2020.09.10'
# date = date.???(".", "\\")
# date

date='2020.09.10'
date = date.replace(".", "\\")
date

'2020\\09\\10'

In [104]:
#biz_name="2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모"
outpath = DOWNLOAD_DIR+"\\{date1}\\{biz_name1}".format(date1=date, biz_name1=biz_name)
outpath

'C:\\mylab\\book\\automate_basic\\download\\2020\\09\\10\\2020년도_제4차_국민생활안전_긴급대응연구사업_신규과제_재공모'

In [ ]:
DOWNLOAD_DIR+"\\*.*"

In [105]:
os.makedirs?

In [107]:
# 경로에 해당하는 폴더를 만든다.
os.makedirs(outpath, exist_ok=True) 

In [164]:
p='C:\\mylab\\book\\automate_basic\\download\\2020\\09\\10'
#!(느낌표) 윈도우즈 os 명령어를 실행할때 사용
!dir {p} /b

2020년도_제4차_국민생활안전_긴급대응연구사업_신규과제_재공모


In [155]:
# download 받은 파일들을 outpath로 이동한다.
#  - download 폴더: C:\\mylab\\book\\automate_basic\\download 하위의 모든 파일
#  - outpath : C:\\mylab\\book\\automate_basic\\download\\2020\\09\\10\\2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고        

In [156]:
DOWNLOAD_DIR

'C:\\mylab\\book\\automate_basic\\download'

In [157]:
# DOWNLOAD_DIR 에 있는 파일을 출력하세요 
for file in glob.glob(DOWNLOAD_DIR+"\\*.*"):
    print(file)

C:\mylab\book\automate_basic\download\(붙임1) 2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모 공고문.hwp
C:\mylab\book\automate_basic\download\(붙임2) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서(양식).zip
C:\mylab\book\automate_basic\download\(붙임3) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서 기타 증빙(양식).hwp
C:\mylab\book\automate_basic\download\(붙임4) 2020년도 국민생활안전 긴급대응연구사업 관련 주요 규정 및 지침.hwp


In [159]:
# 이제 파일을 이동하세요 
for file in glob.glob(DOWNLOAD_DIR+"\\*.*"):
    #print(file)
    shutil.move(file, outpath)

In [158]:
outpath

'C:\\mylab\\book\\automate_basic\\download\\2020\\09\\10\\2020년도_제4차_국민생활안전_긴급대응연구사업_신규과제_재공모'

In [165]:
# 잘 이동 되었는지 확인
# !ls {outpath} -al
!dir {outpath} /b

(붙임1) 2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모 공고문.hwp
(붙임2) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서(양식).zip
(붙임3) 2020년도 국민생활안전 긴급대응연구사업사업 연구개발계획서 기타 증빙(양식).hwp
(붙임4) 2020년도 국민생활안전 긴급대응연구사업 관련 주요 규정 및 지침.hwp


In [166]:
# 존재하는지 체크
os.path.exists(outpath)

True

In [170]:
df

,사업명,공고_담당자,전화번호,공고일,URL,post_id
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773


In [167]:
row.사업명

'2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모'

In [168]:
date = row.공고일.replace(".","//")
date

'2020//09//28'

### refactoring 이전

#### 다운로드 대상을 식별한다.

In [171]:
BOARD_LIST_XL

'C:\\mylab\\book\\automate_basic\\data\\board_list.xlsx'

In [172]:
# 오늘 조회하여 저장한 게시판 목록을 엑셀에서 가져온다
df_today = pd.read_excel(BOARD_LIST_XL, sheet_name='Sheet1') #, index=False
df_today.head()

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961


In [173]:
SEND_HIST_XL

'C:\\mylab\\book\\automate_basic\\data\\mail_send_hsitory.xlsx'

In [175]:
# 어제 보낸 메일 목록을 가져오세요(엑셀에서)
# df_send_hist = pd.???(SEND_HIST_XL) 
df_send_hist = pd.read_excel(SEND_HIST_XL) 

In [176]:
# df_send_hist의 앞부분 5개 를 보시죠
df_send_hist.head()

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
1,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
2,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961
3,2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고,정보통신산업기반과 : 신유진,044-202-6245,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088727
4,2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088726


In [177]:
# 새로 등록된 게시물 만 찾아 보세요
# 다운로드 대상을 식별한다.
# indicator 칼럼 추가 > _merge
# 왜 이렇게 하는가?
# how = 'left'
df_merge = df.merge(df_send_hist, how='left', indicator=True)  

In [178]:
# head를 보시죠
df_merge.head()

,사업명,공고_담당자,전화번호,공고일,URL,post_id,_merge
0,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913,left_only
1,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773,both


In [ ]:
# _merge(indicator) 칼럼에  left_only로 마킹된 애들이 새로 등록된 게시물이에요

In [179]:
# 새로 등록된 게시물 만 필터링해서 저장
# df_merge['_merge']열이 'left_only'인 row만 df_diff에 저장
# df_diff= df_merge[ df_merge['_merge'] ?? '??' ]

df_diff= df_merge[df_merge['_merge'] == 'left_only']
df_diff

,사업명,공고_담당자,전화번호,공고일,URL,post_id,_merge
0,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913,left_only


In [181]:
# _merge 칼럼은 이제 필요 없다. 삭제한다.
df_diff = df_diff.drop(columns=['_merge'])

### refactoring 이후

In [182]:
# 정리를 해보세요.
# 다운로드 대상을 식별한다.
def get_send_df(new_xls_path, send_hist_xls_path):  #함수 정의   
    df = pd.read_excel(new_xls_path, sheet_name='Sheet1') #, index=False
    df_send_hist = pd.read_excel(send_hist_xls_path)     
    
    df_merge = df.merge(df_send_hist, how='left', indicator=True)
    df_diff = df_merge[df_merge['_merge'] == 'left_only']
    df_diff = df_diff.drop(columns=['_merge'])
    return  df_diff

In [ ]:
# df_diff.drop?

In [183]:
BOARD_LIST_XL

'C:\\mylab\\book\\automate_basic\\data\\board_list.xlsx'

In [184]:
SEND_HIST_XL

'C:\\mylab\\book\\automate_basic\\data\\mail_send_hsitory.xlsx'

In [185]:
df_diff = get_send_df(BOARD_LIST_XL, SEND_HIST_XL)

In [186]:
df_diff

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913


In [189]:
import my_util as ut
import time

In [190]:
def mkdir_rm(path0):
    if os.path.exists(path0):
        shutil.rmtree(path0)
    time.sleep(0.5)
    os.makedirs(path0)

In [187]:
for row in df_diff.itertuples(index=True):
    print(row)

Pandas(Index=0, 사업명='2020년도 EU 협력진흥사업 신규과제 공모', 공고_담당자='구주아프리카협력담당관 : 이세원', 전화번호='044-202-4354', 공고일='2020.09.29', URL='https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914', post_id=3114914)
Pandas(Index=1, 사업명='2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모', 공고_담당자='구주아프리카협력담당관 : 이세원', 전화번호='044-202-4354', 공고일='2020.09.29', URL='https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114913', post_id=3114913)


In [191]:
# 새롭게 게시된 게시물만 다운로드 한다.
for row in df_diff.itertuples(index=True):
    driver.get(row.URL) #이동한다
    sleep(2) # 게시판의 내용이 조회 완료되어야 함.  2초 기다린다.
    date = row.공고일.replace(".","\\")    
    biz_name= biz_name.replace(" ","_")    
    outpath = DOWNLOAD_DIR+"\\{date}\\{biz_name}".format(date=date, biz_name=biz_name) # Path to be saved    
    
    element = driver.find_element_by_class_name('down_file')
    element2 = element.find_element_by_class_name('down_btn')
    a_tag_list = element2.find_elements_by_tag_name('a')  
    for a_tag in a_tag_list:
        onclick_script= a_tag.get_attribute('onclick') # onclick 자바스크립트 문장 저장
        if "filePathEncode(" in onclick_script: 
            driver.execute_script(onclick_script, a_tag)  #자바스크립트 실행
            sleep(1)
            
    for file in glob.glob(DOWNLOAD_DIR+"\\*.*"):
        ut.mkdir_rm(outpath)
        shutil.move(file, outpath) # 첨부파일을 outpath로 이동한다. 

## 메일을 보낸다

In [192]:
df_diff

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913


In [ ]:
# 오늘 다운로드 받은 파일들을 사업별로(폴더별로) zip파일을 만든다.

In [ ]:
# 다운로드 폴더를 만든다

In [193]:
date = row.공고일.replace(".", "\\")
print('row.공고일',row.공고일)
print('date',date)

row.공고일 2020.09.29
date 2020\09\29


In [194]:
biz_name= row.사업명.replace(" ","_")  # 폴더에 space가 있으면 프로그램이 오동작하는 경우가 있다.
download_full_path = DOWNLOAD_DIR + "\\{date}\\{biz_name}".format(date=date, biz_name=biz_name)
print(download_full_path)

C:\mylab\book\automate_basic\download\2020\09\29\2020년도_하반기_한-EU_공동연구지원사업_신규과제_공모


In [196]:
# 사업별로 zip파일을 만든다.
from my_util import zip # @@@ zip 처리도 설명해야지.
for row in df_diff.itertuples(index=True):    
    date = row.공고일.replace(".", "\\")
    biz_name= row.사업명.replace(" ","_")
    download_full_path = DOWNLOAD_DIR + "\\{date}\\{biz_name}".format(date=date, biz_name=biz_name)    
    print("원본위치", download_full_path)
    
    file = BASE_DIR+'\\send_temp\\{f_name}.zip'.format(f_name=row.사업명)
    print("압축파일 위치",file)    
    zip(download_full_path, file)    

원본위치 C:\mylab\book\automate_basic\download\2020\09\29\2020년도_EU_협력진흥사업_신규과제_공모
압축파일 위치 C:\mylab\book\automate_basic\send_temp\2020년도 EU 협력진흥사업 신규과제 공모.zip
원본위치 C:\mylab\book\automate_basic\download\2020\09\29\2020년도_하반기_한-EU_공동연구지원사업_신규과제_공모
압축파일 위치 C:\mylab\book\automate_basic\send_temp\2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모.zip


#### 오늘 다운로드한 게시물 목록 엑셀을 송부해야 한다.

In [197]:
NEW_POST_LIST_XL

'C:\\mylab\\book\\automate_basic\\data\\new_post_list.xlsx'

In [198]:
# 새로운 게시물 목록을 엑셀에 저장하세요
# df_diff.to_??(??, encoding='??', index=??) 

df_diff.to_excel(NEW_POST_LIST_XL, encoding='cp949', index=False) 

In [199]:
dest = BASE_DIR+"\\send_temp\\{f_name}".format(f_name='new_post_list.xlsx')
dest

'C:\\mylab\\book\\automate_basic\\send_temp\\new_post_list.xlsx'

In [200]:
# dest(메일 송신 임시 폴더)에 복사하세요
# from shutil import copyfile
# copyfile(??, ??)

from shutil import copyfile
copyfile(NEW_POST_LIST_XL, dest)

'C:\\mylab\\book\\automate_basic\\send_temp\\new_post_list.xlsx'

In [201]:
send_temp_path = BASE_DIR+'\\send_temp'
send_temp_final_path = BASE_DIR+'\\send_temp_final'
print(send_temp_path)
print(send_temp_final_path)

C:\mylab\book\automate_basic\send_temp
C:\mylab\book\automate_basic\send_temp_final


In [202]:
# 무엇이 있나요?
!dir /b C:\mylab\book\automate_basic\send_temp

2020년도 EU 협력진흥사업 신규과제 공모.zip
2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모.zip
new_post_list.xlsx


In [203]:
!dir /b C:\mylab\book\automate_basic\send_temp_final

20201004_171022.zip


In [204]:
#오늘 날짜로 zip파일을 만들어 첨부하세요
#먼저, 오늘 날짜를 만들어 보세요
#strftime @@@ 다양한 사례를 보자
from datetime import datetime
f_name = datetime.now().strftime("%Y%m%d_%H%M%S") #오늘 날짜로 파일명을 만든다
f_name

'20201004_223933'

In [205]:
email_attachment_path = send_temp_final_path + '\\{f_name}.zip'.format(f_name=f_name)
email_attachment_path

'C:\\mylab\\book\\automate_basic\\send_temp_final\\20201004_223933.zip'

In [206]:
# email_attachment_path에 압축파일을 만드세요 
zip(send_temp_path, email_attachment_path) #zip @@@

'ok'

In [207]:
!dir /b C:\mylab\book\automate_basic\send_temp_final

20201004_171022.zip
20201004_223933.zip


In [214]:
emailfrom = "kusung25@gmail.com"
emailto   = "kusung25@naver.com" 
subject = "정보통신부 사업공모 게시판 신규게시물 송신 건"
email_contents_path = 'email_content.txt'

In [212]:
# !cat ./email_content.txt

#### 메일을 보냅니다.
* gmail 설정 방법 설명 필요

In [215]:
# 메일을 보냅니다.
# gmail 설정 방법 설명 필요 @@@
from email_simple import send_mail
rc = None
rc = send_mail(emailto, subject, email_contents_path, email_attachment_path) #@@@ send_mail 세부설명 필요
rc

'ok'

#### 메일을 보낸 후 오늘 처리결과를 반영한다.  board_hsitory
게시물을 추가해 저장한다.

In [216]:
#기존 히스토리
df_hsitory = pd.read_excel(SEND_HIST_XL) 

In [217]:
#기존 히스토리에 오늘 메일 보낸 사업을 추가한다.
# pd.concat 설명한다. @@@
df_hsitory_new = pd.concat([df_diff, df_hsitory], axis=0).reset_index(drop=True)

In [218]:
df_hsitory_new

,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961
5,2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고,정보통신산업기반과 : 신유진,044-202-6245,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088727
6,2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088726
7,2020년도 재난안전플랫폼기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081711
8,2020년도 국민생활안전 긴급대응연구사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081710
9,2020년도 제2차 해외고급과학자초빙사업 Plus (Brain Pool+) 연장공고,미래인재양성과 : 양성녀,044-202-4836,2020.09.01,https://www.msit.go.kr/web/msipContents/conten...,3067835


### 정리를 하면

In [219]:
if rc == "ok":    
    df_hsitory = pd.read_excel(SEND_HIST_XL) 
    df_hsitory_new = pd.concat([df_diff, df_hsitory], axis=0).reset_index(drop=True)
    
    # update history
    df_hsitory_new.to_excel(SEND_HIST_XL, encoding='cp949', index=False) #encoding='cp949' 이건 뭔가?  
    mkdir_rm(send_temp_path) # 임시폴더 삭제
    mkdir_rm(send_temp_final_path)
    print("ok")

PermissionError: [WinError 5] 액세스가 거부되었습니다: 'C:\\mylab\\book\\automate_basic\\send_temp'

## driver.quit()

In [220]:
driver.quit()